In [125]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import model_selection
from sklearn.metrics import make_scorer

In [126]:
df = pd.read_csv('data/AirPass.csv', )
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [127]:
df = df.drop(columns=df.columns[0], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

In [128]:
df.isnull().sum().sum()

310

In [129]:
df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median(), inplace=True)

In [130]:
round(df['Arrival Delay in Minutes'].mean(),2)

15.13

In [131]:
df.groupby(['Gender','satisfaction'])['id'].agg('count')

Gender  satisfaction           
Female  neutral or dissatisfied    30193
        satisfied                  22534
Male    neutral or dissatisfied    28686
        satisfied                  22491
Name: id, dtype: int64

In [132]:
df.groupby(['Type of Travel','satisfaction'])['id'].agg('count')

Type of Travel   satisfaction           
Business travel  neutral or dissatisfied    29909
                 satisfied                  41746
Personal Travel  neutral or dissatisfied    28970
                 satisfied                   3279
Name: id, dtype: int64

In [133]:
df.groupby(['Class','satisfaction'])['id'].agg('count') / df.groupby(['Class'])['id'].agg('count')

Class     satisfaction           
Business  neutral or dissatisfied    0.305749
          satisfied                  0.694251
Eco       neutral or dissatisfied    0.813862
          satisfied                  0.186138
Eco Plus  neutral or dissatisfied    0.753936
          satisfied                  0.246064
Name: id, dtype: float64

In [134]:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  int64  
 2   Customer Type                      103904 non-null  int64  
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  int64  
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

In [136]:
df = pd.get_dummies(df, columns=['Class'])

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 26 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  int64  
 2   Customer Type                      103904 non-null  int64  
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  int64  
 5   Flight Distance                    103904 non-null  int64  
 6   Inflight wifi service              103904 non-null  int64  
 7   Departure/Arrival time convenient  103904 non-null  int64  
 8   Ease of Online booking             103904 non-null  int64  
 9   Gate location                      103904 non-null  int64  
 10  Food and drink                     103904 non-null  int64  
 11  Online boarding                    1039

In [138]:
df.shape[1]

26

In [139]:
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=26)
X_test.shape[0]

20781

In [140]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train, y_train)

StandardScaler()

In [141]:
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [142]:
X_test_sc[0,0]

0.9408251379303

In [143]:
log_reg = linear_model.LogisticRegression()
log_reg.fit(X_train_sc, y_train)
y_pred = log_reg.predict(X_test_sc)
round(metrics.f1_score(y_test, y_pred),3)

0.855

In [144]:
tree_boo = tree.DecisionTreeClassifier(random_state=26)
adaboo = ensemble.AdaBoostClassifier(estimator=tree_boo, random_state=26, learning_rate=0.01)
adaboo.fit(X_train_sc, y_train)
y_pred = adaboo.predict(X_test_sc)

In [145]:
round(metrics.f1_score(y_test, y_pred),3)

0.94

In [146]:
model_for_gs = ensemble.GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = model_selection.GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(metrics.f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.3s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.3s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.5s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.5s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.5s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.0s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.0s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [171]:
model_for_gs = ensemble.GradientBoostingClassifier(learning_rate=1, n_estimators=128)
model_for_gs.fit(X_train_sc, y_train)

GradientBoostingClassifier(learning_rate=1, n_estimators=128)

In [177]:
pd.DataFrame(model_for_gs.feature_importances_, X_test.columns).sort_values([0], ascending=False)

,0
Online boarding,0.435573
Inflight wifi service,0.218206
Type of Travel,0.170401
Class_Business,0.051356
Inflight entertainment,0.036560
Leg room service,0.020496
Customer Type,0.017747
Checkin service,0.010843
id,0.007461
Baggage handling,0.006000


In [147]:
import xgboost

In [148]:
xg = xgboost.XGBClassifier(random_state = 26)
xg.fit(X_train_sc, y_train)
y_pred = xg.predict(X_test_sc)
metrics.f1_score(y_test, y_pred)

0.9573723114544546

In [157]:
xg.feature_importances_

array([0.00608615, 0.00183918, 0.06166838, 0.00541405, 0.1364974 ,
       0.00293267, 0.09319332, 0.00589433, 0.0113934 , 0.01532049,
       0.0027917 , 0.41296762, 0.01654168, 0.03872842, 0.01671294,
       0.01967976, 0.01569425, 0.02891302, 0.01763374, 0.01801952,
       0.00268436, 0.00539673, 0.05860716, 0.0031795 , 0.00221028],
      dtype=float32)

In [169]:
df_feature = pd.DataFrame(xg.feature_importances_, X_test.columns)
df_feature.sort_values(df_feature.columns[0], ascending=False)

,0
Online boarding,0.412968
Type of Travel,0.136497
Inflight wifi service,0.093193
Customer Type,0.061668
Class_Business,0.058607
Inflight entertainment,0.038728
Checkin service,0.028913
Leg room service,0.019680
Cleanliness,0.018020
Inflight service,0.017634
